In [162]:
#подготовка таблицы, добавление и удаление полей

import pandas as pd
import collections
from datetime import date, timedelta
import re
from itertools import chain
from xlrd import open_workbook
import numpy as np


table = pd.read_excel("~\Downloads\Telegram Desktop\db3.xlsx")
table1 = table.copy()

table.drop('Адрес', axis=1, inplace=True)
table.drop('ID', axis=1, inplace=True)


source = table['Source']
source[source == 'ngs'] = 'НГС'
source[source == 'avito'] = 'Avito'
source[source == 'kvadrat54'] = 'Kvadrat54.ru'
source[source == 'Skladoiskatel'] = 'Складоискатель'

table.drop('Source', axis=1, inplace=True)
table.drop('UniqueID', axis=1, inplace=True)


table.insert(0, 'Name', None)
table['Name'] = table['name']
table.drop('name', axis=1, inplace=True)
table = table.rename(index=str, columns={"Name": "name", "Street": "street", "House": "house", "City": "city"})

table['street'] = table['street'].astype(str)
table['house'] = table['house'].astype(str)



table.insert(1, 'coordinates', None)
table['coordinates'] = table['pos']
table.drop('pos', axis=1, inplace=True)

table.drop('свободный столбец (сохранение всех ссылок на повторы)', axis=1, inplace=True)


cols = table.columns.tolist()
new_cols = cols[:cols.index('operator_name')+1] #таблица
new_cols1 = cols[cols.index('operator_name')+1:] #листинги
table1 = table[new_cols1]
table = table[new_cols]


cols = table.columns.tolist()
cols.remove('listings')
cols.extend(['listings'])
table = table[cols]


table.to_excel('db.xlsx')

C:\Users\Константин\Anaconda3\envs\py35\lib\site-packages\ipykernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Константин\Anaconda3\envs\py35\lib\site-packages\ipykernel\__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Константин\Anaconda3\envs\py35\lib\site-packages\ipykernel\__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Константин\Anaconda3\envs\py35\lib\site-packages\ipykernel\__main__.py:24: SettingWithCopyWarning

In [159]:
#создание json-файла

book = open_workbook('db.xlsx')
sheet = book.sheet_by_index(0)

keys = [sheet.cell(0, col_index).value for col_index in range(sheet.ncols)]

dict_list = []
for row_index in range(1, sheet.nrows):
    d = collections.OrderedDict([(keys[col_index],sheet.cell(row_index, col_index).value )
                     for col_index in range(sheet.ncols)])
    dict_list.append(d)

table2 = table1.loc[:, ~table1.columns.str.contains('^Unnamed')]
links = table2.filter(regex=("List1_link"))
dates = table2.filter(regex=("date"))
prices = table2.filter(regex=("price"))

nrows, ncols = table1.shape
ncols = int(ncols/3)
unav = [np.nan, 'null', pd.NaT]
for row in range(nrows):
    listings = []
    for col in range(ncols):
        if (links.iloc[row,col] not in unav) and (dates.iloc[row,col] not in unav):
            date = np.nan
            sourc = np.nan
            price = np.nan
            if (dates.iloc[row,col] not in unav):
                date = '20'+dates.iloc[row,col].strftime('%y-%m-%d')+'T00:00:00.000Z'
            if (source[row] not in unav):    
                sourc = source[row]
            if (prices.iloc[row,col] not in unav):    
                price = prices.iloc[row, col]
            link = links.iloc[row,col]
            listings.append(collections.OrderedDict([("date", {"$date": date}), ("source", sourc),
                                                     ("price", price), ("link", link)]))
        else: 
            continue
    dict_list[row].popitem(last=False)        
    coord = dict_list[row]['coordinates'].split(' ')
    dict_list[row]['coordinates'] = [float(coord[1]), float(coord[0])]
    dict_list[row]['listings'] = listings
    dict_list[row]['latest_listing'] = 'null'
    if (dates.iloc[row,0] not in unav):
        dict_list[row]['latest_listing'] = {"$date": '20' + dates.iloc[row, 0].strftime('%y-%m-%d')+'T00:00:00.000Z'}
    dict_list[row].move_to_end('listings')
    
for entry in dict_list:
    for key in entry.keys():
        if (key == 'website') and (entry[key] not in unav) and ('http' not in str(entry[key])):    
            entry[key] = 'http://' + str(entry[key])
        if entry[key] == 'null':   
            entry[key] = np.nan
        if entry[key] == 'true':
            entry[key] = True
        if entry[key] == 'false':
            entry[key] == False    

0
0
1
0
2
0
3
0
4
0
5
0
6
0
7
0
8
0
9
0
10
0
11
0
12
0
13
0
14
0
15
0
16
0
17
0
18
0
19
0
20
0
21
0
22
0
23
0
24
0
25
0
26
0
27
0
28
0
29
0
30
0
31
0
32
0
33
0
34
0
35
0
36
0
37
0
38
0
39
0
40
0
41
0
42
0
43
0
44
0
45
0
46
0
47
0
48
0
49
0
50
0
51
0
52
0
53
0
54
0
55
0
56
0
57
0
58
0
59
0
60
0
61
0
62
0
63
0
64
0
65
0
66
0
67
0
68
0
69
0
70
0
71
0
72
0
73
0
74
0
75
0
76
0
77
0
78
0
79
0
80
0
81
0
82
0
83
0
84
0
85
0
86
0
87
0
88
0
89
0
90
0
91
0
92
0
93
0
94
0
95
0
96
0
97
0
98
0
99
0
100
0
101
0
102
0
103
0
104
0
105
0
106
0
107
0
108
0
109
0
110
0
111
0
112
0
113
0
114
0
115
0
116
0
117
0
118
0
119
0
120
0
121
0
122
0
123
0
124
0
125
0
126
0
127
0
128
0
129
0
130
0
131
0
132
0
133
0
134
0
135
0
136
0
137
0
138
0
139
0
140
0
141
0
142
0
143
0
144
0
145
0
146
0
147
0
148
0
149
0
150
0
151
0
152
0
153
0
154
0
155
0
156
0
157
0
158
0
159
0
160
0
161
0
162
0
163
0
164
0
165
0
166
0
167
0
168
0
169
0
170
0
171
0
172
0
173
0
174
0
175
0
176
0
177
0
178
0
1
179
0
180
0
181
0
182
0
183
0
184


In [161]:
#выгрузка в файл
import json
import simplejson

h = json.dumps(dict_list, ensure_ascii=False)
js = json.loads(h, object_pairs_hook=collections.OrderedDict)
with open('data.json', 'w', encoding = 'utf-8') as outfile:
    simplejson.dump(js, outfile, indent=2, ignore_nan=True, ensure_ascii=False)